In [11]:
import openai
import os

openai.api_key  = os.environ.get('OPENAI_API_KEY')

In [12]:
import json

def get_current_weather(location, unit="fahrenheit"):
    # It is supposed to be a real API call here
    # But we will just return a dummy response
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }

    # json.dumps() function will convert a subset of Python objects into a json string.
    return json.dumps(weather_info)

In [13]:
functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": [
              "celsius",
              "fahrenheit"
            ]
          }
        },
        "required": [
          "location"
        ]
      }
    }
  ]

In [14]:
messages = [
    {
        "role":  "user",
        "content": "What's the weather like in Boston?"
    }
]

In [15]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)
response

<OpenAIObject chat.completion id=chatcmpl-8FDnREjcFQgBSvhJjyhMOQU2f7APa at 0x107af2120> JSON: {
  "id": "chatcmpl-8FDnREjcFQgBSvhJjyhMOQU2f7APa",
  "object": "chat.completion",
  "created": 1698638645,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n\"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 17,
    "total_tokens": 99
  }
}

In [16]:
messages.append(response.choices[0]["message"])
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 <OpenAIObject at 0x107b22da0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n\"location\": \"Boston, MA\"\n}"
   }
 }]

Our task is to execute get_current_weather with the provided arguments. OpenAI does not execute the function. Instead, it’s the job of our chatbot to run it and parse the returned data.

In [17]:
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": get_current_weather("Boston")
})

In [18]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)
response.choices[0]["message"]["content"]

'The current weather in Boston is 72°F and sunny, but it is also windy.'